### Import All mandatory Libraries

In [49]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV

In [50]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Data Cleaning and Preprocessing

In [1]:
# df1 = 'data/sentiment_analysis.csv'
# em1 = pd.read_csv(df1)
# em1.shape

NameError: name 'pd' is not defined

In [42]:
# df2 = 'data/test.csv'
# em2 = pd.read_csv(df2, encoding='unicode_escape')
# em2 = em2[(em2['sentiment']=='positive') | (em2['sentiment']=='neutral') | (em2['sentiment']=='negative')]
# em2.shape

In [62]:
import os
import pandas as pd

# File path for the existing dataset
input_file1 = 'data/sentimental_data.csv'

# Read data from CSV file
em1 = pd.read_csv(input_file1)

# Select relevant columns
# em1 = em1[['text', 'sentiment']]
em1 = em1[(em1['sentiment']=='positive') | (em1['sentiment']=='neutral') | (em1['sentiment']=='negative')]
# Display the dataset
print(em1.shape)  # Display the first few rows of em1
print(em2.shape)  # Display the first few rows of em1


(2118, 2)
(2118, 2)


#### Concatination of 2 Dataframes

In [63]:
em1 = em1[['text','sentiment']]
# em2 = em2[['text','sentiment']]
# final_data = pd.concat([em1,em2],axis=0)
final_data = em1
final_data.shape


(2118, 2)

In [45]:
# # ----------- This Shell is Experimetal can not harm the model if mistakenly run only this is for elaboration --------------
# Mapping of positive negative and neutral
sentiment_to_label = {'positive': 2, 'neutral': 1, 'negative': 0}
# Apply the mapping to create a new 'label' column
final_data['label'] = final_data['sentiment'].map(sentiment_to_label)
final_data.head()

,text,sentiment,label
0,I am not happy,negative,0.0
1,I love the new design of the website!,positive,2.0
2,The weather is nice today.,neutral,1.0
3,I'm really disappointed with the service.,negative,0.0
4,"The movie was fantastic, I enjoyed every momen...",positive,2.0


In [65]:
# We Only Have to work With text and its sentiment we can also go with it's label as well only some will change in the code

data = final_data[['text','sentiment']]
# for Experiment
# data = emotions[['text','label']]
data.shape

(2118, 2)

### Feature Extraction

In [66]:
tfidf = TfidfVectorizer(
    use_idf=True,
    norm='l2',
    smooth_idf=True
)
# y= data.label.values
y = data.sentiment.values
X = tfidf.fit_transform(data['text'].values.astype('U'))
tfidf_vectorizer = open('tfidf_vectorizer.sav','wb')
pickle.dump(tfidf,tfidf_vectorizer)
tfidf_vectorizer.close()

In [55]:
# Load your dataset into a pandas DataFrame
# data = pd.read_csv('data/sentimental_data.csv')

# # Initialize TF-IDF Vectorizer
# tfidf = TfidfVectorizer(
#     use_idf=True,
#     norm='l2',
#     smooth_idf=True
# )

# # Fit and transform TF-IDF on the text data
# X = tfidf.fit_transform(data['text'].values.astype('U'))  # Convert to Unicode strings
# y = data['sentiment'].values  # Assuming 'sentiment' is the column name for labels

# # Save the TF-IDF vectorizer using pickle
# with open('tfidf_vectorizer.sav', 'wb') as file:
#     pickle.dump(tfidf, file)

# print("TF-IDF vectorizer saved successfully.")

TF-IDF vectorizer saved successfully.


### Prepare data for Traning and Testing

In [73]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,test_size=0.2,shuffle=False)

### Model Traning and Save Model

In [74]:
clf = LogisticRegressionCV(
    cv=5,
    scoring='accuracy',
    random_state=0,
    n_jobs=-1,
    verbose=3,
    max_iter=300
).fit(X_train,y_train)
saved_model = open('saved_model.sav','wb')
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.9s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


### Check Model Accuracy

In [75]:
filename = 'saved_model.sav'
with open(filename, 'rb') as model_file:
    saved_model = pickle.load(model_file)
accuracy = saved_model.score(X_test,y_test)
print(f"Model Accuracy is : {accuracy*100:.2f}%")

Model Accuracy is : 100.00%


### Prediction

##### You Can Directly run Prediction shell 

In [83]:
import pickle

filename = 'saved_model.sav'
with open(filename, 'rb') as model_file:
    saved_model = pickle.load(model_file)
    
vectorizer_filename = 'tfidf_vectorizer.sav'
with open(vectorizer_filename, 'rb') as vectorizer_file:
    tfidf_vectorizer = pickle.load(vectorizer_file)

text = input("Enter you Text to Predict the Sentiment :")
text_tfidf = tfidf_vectorizer.transform([text])
prediction = saved_model.predict(text_tfidf)
print(f"Your Text Sentiment is : {prediction[0]}")

Enter you Text to Predict the Sentiment : you are not good


Your Text Sentiment is : negative
